In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [2]:
TRAINING_DIR = 'F:/Xray_Gaus/train'
TEST_DIR='F:/Xray_Gaus/test'
VAL_DIR='F:/Xray_Gaus/val'


In [3]:
num_classes = 2
IMG_SIZE = 224
BATCH_SIZE = 32
input_shape = (224,224, 3)

learning_rate = 0.001
weight_decay = 0.0001
num_epochs = 10

In [4]:
from keras_preprocessing.image import ImageDataGenerator

# Training IDG
train_idg = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1,
    horizontal_flip=True)

# Training Gen
train_gen = train_idg.flow_from_directory(
    TRAINING_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='categorical',
    classes=['normal_175_2','pneumonia_175','COVID-19'],
    subset='training'
) 

Found 526 images belonging to 3 classes.


In [5]:
val_idg = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1,
    horizontal_flip=True)

# Training Gen
val_gen = val_idg.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='categorical',
    classes=['normal_20_2','pneumonia_20','COVID-19'],
    subset='training'
)

Found 60 images belonging to 3 classes.


In [6]:
test_idg = ImageDataGenerator(rescale=1./255)

# Test Gen
test_gen = test_idg.flow_from_directory(
        TEST_DIR,
        target_size=(IMG_SIZE, IMG_SIZE),
        shuffle=True,
        class_mode='categorical',
    classes=['normal_20_2','pneumonia_20','COVID-19'],
        #subset='validation'
)

Found 60 images belonging to 3 classes.


In [7]:
new_input = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

from tensorflow.keras.optimizers import Adam

base_model = keras.applications.DenseNet169(
    weights = 'imagenet',
    #input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,input_tensor=new_input)

base_model.trainable = False

x = base_model.output
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Dense(2048, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Dense(2048, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(3, activation='softmax')(x)

# se acopla el modelo
model_1 = keras.Model(base_model.input, outputs)

# congelar capas
for layer in base_model.layers:
    layer.trainable = False

# compilar el modelo.
model_1.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')]
        )

model_1.summary()

___________
conv5_block21_0_relu (Activatio (None, 7, 7, 1280)   0           conv5_block21_0_bn[0][0]         
__________________________________________________________________________________________________
conv5_block21_1_conv (Conv2D)   (None, 7, 7, 128)    163840      conv5_block21_0_relu[0][0]       
__________________________________________________________________________________________________
conv5_block21_1_bn (BatchNormal (None, 7, 7, 128)    512         conv5_block21_1_conv[0][0]       
__________________________________________________________________________________________________
conv5_block21_1_relu (Activatio (None, 7, 7, 128)    0           conv5_block21_1_bn[0][0]         
__________________________________________________________________________________________________
conv5_block21_2_conv (Conv2D)   (None, 7, 7, 32)     36864       conv5_block21_1_relu[0][0]       
_________________________________________________________________________________________________

In [8]:
#inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))


base_model1 = keras.applications.VGG19(
    weights = 'imagenet',
    #input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,input_tensor=new_input)

base_model1.trainable = False

x1 = base_model1.output
x1 = keras.layers.Flatten()(x1)
x1 = keras.layers.Dropout(0.4)(x1)
x1 = keras.layers.Dense(2048, activation='relu')(x1)
x1 = keras.layers.BatchNormalization()(x1)
x1 = keras.layers.Dropout(0.4)(x1)
x1 = keras.layers.Dense(2048, activation='relu')(x1)
x1 = keras.layers.BatchNormalization()(x1)
x1 = keras.layers.Dropout(0.2)(x1)
outputs = keras.layers.Dense(3, activation='softmax')(x1)

# se acopla el modelo
model_2 = keras.Model(base_model1.input, outputs)

# congelar capas
for layer in base_model1.layers:
    layer.trainable = False

# compilar el modelo.
model_2.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')]
        )

model_2.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [9]:
base_model2 = keras.applications.InceptionV3(
    weights = 'imagenet',
    #input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,input_tensor=new_input)

base_model2.trainable = False

x2 = base_model2.output
x2 = keras.layers.Flatten()(x2)
x2 = keras.layers.Dropout(0.4)(x2)
x2 = keras.layers.Dense(2048, activation='relu')(x2)
x2 = keras.layers.BatchNormalization()(x2)
x2 = keras.layers.Dropout(0.4)(x2)
x2 = keras.layers.Dense(2048, activation='relu')(x2)
x2 = keras.layers.BatchNormalization()(x2)
x2 = keras.layers.Dropout(0.2)(x2)
outputs = keras.layers.Dense(3, activation='softmax')(x2)

# se acopla el modelo
model_3 = keras.Model(base_model2.input, outputs)

# congelar capas
for layer in base_model2.layers:
    layer.trainable = False

# compilar el modelo.
model_3.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')]
        )

model_3.summary()

[0][0]     
__________________________________________________________________________________________________
mixed7 (Concatenate)            (None, 12, 12, 768)  0           activation_60[0][0]              
                                                                 activation_63[0][0]              
                                                                 activation_68[0][0]              
                                                                 activation_69[0][0]              
__________________________________________________________________________________________________
conv2d_72 (Conv2D)              (None, 12, 12, 192)  147456      mixed7[0][0]                     
__________________________________________________________________________________________________
batch_normalization_76 (BatchNo (None, 12, 12, 192)  576         conv2d_72[0][0]                  
_________________________________________________________________________________________________

In [10]:
mergedOutput = tf.keras.layers.Concatenate()([model_1.output,model_2.output,model_3.output])

hidden = tf.keras.layers.Dense(9, activation='relu')(mergedOutput)
output = tf.keras.layers.Dense(3, activation='softmax')(hidden)
model = tf.keras.Model(inputs=new_input, outputs=output)
                # plot graph of ensemble
            #plot_model(model, show_shapes=True, to_file='model_graph.png')
                # compile
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


___________
batch_normalization_88 (BatchNo (None, 5, 5, 192)    576         conv2d_84[0][0]                  
__________________________________________________________________________________________________
conv5_block31_1_bn (BatchNormal (None, 7, 7, 128)    512         conv5_block31_1_conv[0][0]       
__________________________________________________________________________________________________
block3_conv3 (Conv2D)           (None, 56, 56, 256)  590080      block3_conv2[0][0]               
__________________________________________________________________________________________________
activation_76 (Activation)      (None, 5, 5, 320)    0           batch_normalization_80[0][0]     
__________________________________________________________________________________________________
mixed9_0 (Concatenate)          (None, 5, 5, 768)    0           activation_78[0][0]              
                                                                 activation_79[0][0]             

In [11]:
STEP_SIZE_TRAIN = train_gen.n // train_gen.batch_size
STEP_SIZE_VALID = val_gen.n // val_gen.batch_size

history=model.fit(x = train_gen,
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_data = val_gen,
          validation_steps = STEP_SIZE_VALID,
          epochs = 5)

Epoch 1/5
16/16 [==============================] - 311s 18s/step - loss: 1.0631 - accuracy: 0.4696 - val_loss: 1.1487 - val_accuracy: 0.4062
Epoch 2/5
16/16 [==============================] - 224s 14s/step - loss: 1.0016 - accuracy: 0.5312 - val_loss: 1.2828 - val_accuracy: 0.2812
Epoch 3/5
16/16 [==============================] - 380s 24s/step - loss: 0.9371 - accuracy: 0.5732 - val_loss: 1.1581 - val_accuracy: 0.4062
Epoch 4/5
16/16 [==============================] - 173s 11s/step - loss: 0.9204 - accuracy: 0.5532 - val_loss: 1.1301 - val_accuracy: 0.2812
Epoch 5/5
16/16 [==============================] - 226s 15s/step - loss: 0.8725 - accuracy: 0.7333 - val_loss: 1.2201 - val_accuracy: 0.3438


In [12]:
results = model.evaluate(test_gen, batch_size=10)

2/2 [==============================] - 25s 12s/step - loss: 1.1783 - accuracy: 0.3833


In [13]:
model.save('F:/X_ray_Models/e2_class3_xray_model_174_2')

history.history

INFO:tensorflow:Assets written to: F:/X_ray_Models/e2_class3_xray_model_174_2\assets


{'loss': [1.0231479406356812,
  0.9676163196563721,
  0.9238491654396057,
  0.9090377688407898,
  0.8692166209220886],
 'accuracy': [0.4898785352706909,
  0.5809716582298279,
  0.5850202441215515,
  0.6113360524177551,
  0.73886638879776],
 'val_loss': [1.1487349271774292,
  1.2827742099761963,
  1.1580848693847656,
  1.1300809383392334,
  1.220134973526001],
 'val_accuracy': [0.40625, 0.28125, 0.40625, 0.28125, 0.34375]}

In [14]:
import numpy as np
y_pred = model.predict(test_gen, verbose=1)
y_pred = np.argmax(y_pred, axis = 1)

2/2 [==============================] - 33s 9s/step


In [15]:
y_true=test_gen.labels

In [16]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.32      0.90      0.47        20
           1       0.50      0.05      0.09        20
           2       0.00      0.00      0.00        20

    accuracy                           0.32        60
   macro avg       0.27      0.32      0.19        60
weighted avg       0.27      0.32      0.19        60



In [17]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, y_pred)

array([[18,  1,  1],
       [18,  1,  1],
       [20,  0,  0]], dtype=int64)